In [25]:
import pandas as pd
import numpy as np
import psycopg2
import uuid

#Leer el archivo
df = pd.read_excel("TCA CHILE.xlsx")

#Dejar columnas vacias como 0
df = df.replace({None: 0})
df = df.fillna(0)
df = df.replace({',': 0})   
df = df.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

#Renombrar columnas para que coincidan con la tabla
df.rename(columns={
    'Codigo ': 'id',
    'Nombre': 'nombre',
    'Energia': 'energia',
    'Cenizas': 'cenizas',
    'Proteinas': 'proteinas',
    'H de C disp.': 'hidratos_carbono',
    'Azucares Totales': 'azucares_totales',
    'Fibra Dietetica Total': 'fibra_dietetica',
    'Lipidos Totales': 'lipidos_totales',
    'Ac grasos sat': 'acidos_grasos_saturados',
    'Ac grasos monoinsat': 'acidos_grasos_monoinsaturados',
    'Ac grasos poliinsat': 'acidos_grasos_poliinsaturados',
    'Ac grasos trans': 'acidos_grasos_trans',
    'Colesterol': 'colesterol',
    'Vitamina A': 'vitamina_A',
    'Vit C': 'vitamina_C',
    'Vit D': 'vitamina_D',
    'Vit E': 'vitamina_E',
    'Vit K': 'vitamina_K',
    'Vit B1': 'vitamina_B1',
    'Vit B2': 'vitamina_B2',
    'Niancina': 'niacina',
    'Vit B6': 'vitamina_B6',
    'Ac Pantotenico': 'acido_pantotenico',
    'Vit B12': 'vitamina_B12',
    'Folatos': 'folatos',
    'Sodio ': 'sodio',
    'Potasio ': 'potasio',
    'Calcio ': 'calcio',
    'Fosforo': 'fosforo',
    'Magnesio': 'magnesio',
    'Hierro': 'hierro',
    'Zinc': 'zinc',
    'Cobre': 'cobre',
    'Selenio': 'selenio',
    'Alcohol': 'alcohol',
    'Humedad ': 'humedad',
    'Categoria ': 'categoria_id'
}, inplace=True)

#Limpiar la categoría
df['categoria_id'] = df['categoria_id'].astype(str).str.replace('2.', '', regex=False)

#Generar ID nuevo (UUID) para cada fila
df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]
df['energia'] = df['energia'].astype(float).round(2)

for col in df.columns:
    if col not in ['id', 'nombre', 'categoria_id']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        df[col] = df[col].round(2)
        
#Insertar en PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    dbname="BD_SAFETYIRC",
    user="postgres",
    password="1234"
)
cursor = conn.cursor()
#Preparar query SQL
sql = """
INSERT INTO api_alimento (
    id, nombre, energia, cenizas, proteinas, hidratos_carbono,
    azucares_totales, fibra_dietetica, lipidos_totales, acidos_grasos_saturados,
    acidos_grasos_monoinsaturados, acidos_grasos_poliinsaturados, acidos_grasos_trans,
    colesterol, "vitamina_A", "vitamina_C", "vitamina_D", "vitamina_E", "vitamina_K",
    "vitamina_B1", "vitamina_B2", niacina, "vitamina_B6", acido_pantotenico,
    "vitamina_B12", folatos, sodio, potasio, calcio, fosforo, magnesio,
    hierro, zinc, cobre, selenio, alcohol, activo, categoria_id, humedad
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
          %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

#Insertar
for _, row in df.iterrows():
    values = (
        row['id'],
        row['nombre'],
        row['energia'],
        row['cenizas'],
        row['proteinas'],
        row['hidratos_carbono'],
        row['azucares_totales'],
        row['fibra_dietetica'],
        row['lipidos_totales'],
        row['acidos_grasos_saturados'],
        row['acidos_grasos_monoinsaturados'],
        row['acidos_grasos_poliinsaturados'],
        row['acidos_grasos_trans'],
        row['colesterol'],
        row['vitamina_A'],
        row['vitamina_C'],
        row['vitamina_D'],
        row['vitamina_E'],
        row['vitamina_K'],
        row['vitamina_B1'],
        row['vitamina_B2'],
        row['niacina'],
        row['vitamina_B6'],
        row['acido_pantotenico'],
        row['vitamina_B12'],
        row['folatos'],
        row['sodio'],
        row['potasio'],
        row['calcio'],
        row['fosforo'],
        row['magnesio'],
        row['hierro'],
        row['zinc'],
        row['cobre'],
        row['selenio'],
        row['alcohol'],
        True,  # Activo por defecto
        int(row['categoria_id']) if row['categoria_id'] not in [None, ''] else None,
        row['humedad']
    )
    cursor.execute(sql, values)

conn.commit()
cursor.close()
conn.close()

print("Carga completa a BD.")


C:\Users\nicol\AppData\Local\Temp\ipykernel_12908\3982682096.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({',': 0})
C:\Users\nicol\AppData\Local\Temp\ipykernel_12908\3982682096.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


Carga completa a BD.


In [13]:
import pandas as pd

df = pd.read_excel("TCA CHILE.xlsx")
df.rename(columns={
    'Codigo ': 'id',
    'Nombre': 'nombre',
    'Energia': 'energia',
    'Cenizas': 'cenizas',
    'Proteinas': 'proteinas',
    'H de C disp.': 'hidratos_carbono',
    'Azucares Totales': 'azucares_totales',
    'Fibra Dietetica Total': 'fibra_dietetica',
    'Lipidos Totales': 'lipidos_totales',
    'Ac grasos sat': 'acidos_grasos_saturados',
    'Ac grasos monoinsat': 'acidos_grasos_monoinsaturados',
    'Ac grasos poliinsat': 'acidos_grasos_poliinsaturados',
    'Ac grasos trans': 'acidos_grasos_trans',
    'Colesterol': 'colesterol',
    'Vitamina A': '"vitamina_A"',
    'Vit C': 'vitamina_C',
    'Vit D': 'vitamina_D',
    'Vit E': 'vitamina_E',
    'Vit K': 'vitamina_K',
    'Vit B1': 'vitamina_B1',
    'Vit B2': 'vitamina_B2',
    'Niacina': 'niacina',
    'Vit B6': 'vitamina_B6',
    'Ac Pantotenico': 'acido_pantotenico',
    'Vit B12': 'vitamina_B12',
    'Folatos': 'folatos',
    'Sodio ': 'sodio',
    'Potasio ': 'potasio',
    'Calcio ': 'calcio',
    'Fosforo': 'fosforo',
    'Magnesio': 'magnesio',
    'Hierro': 'hierro',
    'Zinc': 'zinc',
    'Cobre': 'cobre',
    'Selenio': 'selenio',
    'Alcohol': 'alcohol',
    'Humedad ': 'humedad',
    'Categoria ': 'categoria_id'
}, inplace=True)
#mostrar columnas
print(df.columns.tolist())

['id', 'categoria_id', 'nombre', 'energia', 'humedad', 'cenizas', 'proteinas', 'hidratos_carbono', 'azucares_totales', 'fibra_dietetica', 'lipidos_totales', 'acidos_grasos_saturados', 'acidos_grasos_monoinsaturados', 'acidos_grasos_poliinsaturados', 'acidos_grasos_trans', 'colesterol', '"vitamina_A"', 'vitamina_C', 'vitamina_D', 'vitamina_E', 'vitamina_K', 'vitamina_B1', 'vitamina_B2', 'Niancina', 'vitamina_B6', 'acido_pantotenico', 'vitamina_B12', 'folatos', 'sodio', 'potasio', 'calcio', 'fosforo', 'magnesio', 'hierro', 'zinc', 'cobre', 'selenio', 'alcohol']


In [13]:
import pandas as pd
import numpy as np
import psycopg2
#conectar a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="BD_SAFETYIRC",
    user="postgres",
    password="1234",
    port="5432"
)
#crear un cursor
cursor = conn.cursor()


df = pd.read_excel('TCA CHILE.xlsx', sheet_name='Cod por Categoria')
#renombrear las columnas
df.columns = ['Codigo', 'Categorias']
#si codigo es 2.20 dejar 20
df['Codigo'] = df['Codigo'].astype(str).str.replace('2.', '', regex=False)
#si categoria es verduras Codigo sea 10
df.loc[df['Categorias'] == 'Verduras', 'Codigo'] = 10
df.loc[df['Categorias'] == 'Salsas', 'Codigo'] = 20
#truncar tabla y restablecer los id
cursor.execute("TRUNCATE TABLE api_categoriaalimento RESTART IDENTITY CASCADE;")
#subir valores a la base de datos
for index, row in df.iterrows():
    cursor.execute(
        "INSERT INTO api_categoriaalimento (id, nombre) VALUES (%s, %s)",
        (row['Codigo'], row['Categorias'])
    )
#guardar los cambios
conn.commit()
